In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
from functools import partial
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.distributions as dist
import pyro
import pyro.distributions as pyro_dist
from pyro.infer.autoguide import AutoDiagonalNormal, AutoMultivariateNormal, init_to_mean, AutoNormal
from pyro.infer import SVI, Trace_ELBO

import bayesreg_pyro as br
import vi_diagnostics as vid

import sys
# append path to PSIS, cloned from https://github.com/avehtari/PSIS
sys.path.append("/Users/zizi/code/SML_VI")
from PSIS.py import psis

In [3]:
data = pd.read_csv("bodyfat.csv").drop("Density", axis=1)
data = data - data.mean()

y_data = torch.Tensor(data["Bodyfat"].values)
x_data = torch.Tensor(data.drop("Bodyfat", axis=1).values)
in_features = x_data.shape[1]

In [4]:
model = br.BayesianRegression(in_features, 1)
guide = AutoDiagonalNormal(model) # Stochastic Mean field
adam = pyro.optim.Adam({"lr": 0.01})
svi = SVI(model, guide, adam, loss=Trace_ELBO())

torch.manual_seed(12)
pyro.clear_param_store()
tic=time.time()
elbos, samples = br.run_vi(x_data, y_data, svi, model, guide)
toc=time.time()
print(f"SVI time: {toc-tic} sec")

SVI time: 21.97310996055603 sec


In [20]:
imp_lw = vid.sample_and_calculate_log_impweight(x_data, y_data, model, guide, 200)
lw, kappa = psis.psislw(imp_lw)
print(kappa)

0.808535623550415


In [21]:
torch.manual_seed(12)
pyro.clear_param_store()

guide_mvt = AutoMultivariateNormal(model, init_loc_fn=init_to_mean)
svi_mvt = SVI(model, guide_mvt, pyro.optim.Adam({"lr": 0.01}), loss=Trace_ELBO())

tic = time.time()
elbos_mvt, samples_mvt = br.run_vi(x_data, y_data, svi_mvt, model, guide_mvt)
toc=time.time()
print(f"Non-MFVI time: {toc-tic} sec")

Non-MFVI time: 23.327373027801514 sec


In [29]:
imp_lw = vid.sample_and_calculate_log_impweight(x_data, y_data, model, guide_mvt, 200)
lw, kappa = psis.psislw(imp_lw)
print(kappa)

0.2802536725997925
